In [92]:
import pandas as pd

#On s'occupe d'abord de nettoyer les données sur le nombre de validation par jour 
%config NotebookApp.iopub_data_rate_limit=100000000000.0
url2 = 'validations-reseau-ferre-nombre-validations-par-jour-1er-semestre.csv'
données_validation2= pd.read_csv(url2)
données_validation2 = pd.read_csv(url2, delimiter=';')

#On extrait les données que pour le mois de mai, à défaut d'avoir celles du mois de juillet et on fait la moyenne sur 1 jour
données_validation2['JOUR'] = pd.to_datetime(données_validation2['JOUR'])
somme_par_station = données_validation2.groupby('LIBELLE_ARRET')['NB_VALD'].sum()
resultat_final = pd.DataFrame({'Station': somme_par_station.index, 'Nombre de validations': somme_par_station.values})
resultat_final['Nombre de validations'] = resultat_final['Nombre de validations'].div(120)


#On s'occupe ensuite de nettoyer les données sur la répartition de l'affluence sur une journée
%config NotebookApp.iopub_data_rate_limit=100000000000.0
url = 'validations-reseau-ferre-profils-horaires-par-jour-type-1er-semestre (1).csv'
données_validation= pd.read_csv(url)
données_validation = pd.read_csv(url, delimiter=';')
données_validation_station_horaire = données_validation[['LIBELLE_ARRET', 'TRNC_HORR_60','pourc_validations']]
donnees_organisees = données_validation_station_horaire.pivot_table(index='LIBELLE_ARRET', columns='TRNC_HORR_60', values='pourc_validations', aggfunc='mean').reset_index()

#On fusionne les données des deux tableaux pour avoir le nombre de validation sur chaque plage horaire
merged_df = pd.merge(resultat_final, donnees_organisees, left_on='Station', right_on='LIBELLE_ARRET')
columns_to_multiply = donnees_organisees.columns[1:]
resultat = merged_df.copy()
resultat[columns_to_multiply] = resultat[columns_to_multiply].multiply(resultat['Nombre de validations'], axis=0)
resultat[columns_to_multiply] = resultat[columns_to_multiply].div(100)
resultat = resultat.iloc[:, 2:]
print(resultat)

       LIBELLE_ARRET       0H-1H     10H-11H     11H-12H     12H-13H  \
0           ABBESSES  115.144828  228.601668  287.922354  341.937938   
1              ABLON    0.760772  107.806447  103.667848  101.801420   
2      ACHERES GRAND    0.096100    3.608555    4.558343    4.380558   
3      ACHERES VILLE   13.156728  371.771529  385.178860  399.964516   
4      AEROPORT CDG1   11.241080  365.959588  400.218116  464.631286   
..               ...         ...         ...         ...         ...   
742           VOSVES         NaN    1.766890    1.709345    3.796382   
743  VULAINES SUR SE         NaN    5.526653    3.634047    4.388368   
744           WAGRAM   60.803640  404.933144  471.944486  556.146386   
745           YERRES    6.759920  341.263295  348.811872  331.236080   
746   YVRIS NOISY GD    0.000000   36.692990   42.179098   39.562418   

        13H-14H     14H-15H     15H-16H     16H-17H     17H-18H  ...  \
0    363.037776  423.081884  489.998512  543.290673  578.979541